In [ ]:
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.io import push_notebook

output_notebook()
        
jscode="""
        // Define a callback to capture errors on the Python side
        function callback(msg){
            console.log("Python callback returned unexpected message:", msg)
        }
        callbacks = {iopub: {output: callback}};

        // Generate a command to execute in Python
        var ranges = {x0: x_range.attributes.start,
                      y0: y_range.attributes.start,
                      x1: x_range.attributes.end,
                      y1: y_range.attributes.end}
        var range_str = JSON.stringify(ranges)
        var cmd = "%s(" + range_str + ")"
        
        // Execute the command on the Python kernel
        var kernel = IPython.notebook.kernel;
        kernel.execute(cmd, callbacks, {silent : false});
"""

p = figure(plot_width=400, plot_height=400, x_range=(0, 10), y_range=(0, 10))

def normalize_points(ranges):
    """
    Normalizes the points into the current range and
    pushes the updated data via push_notebook.
    """
    xvals = np.linspace(ranges['x0'], ranges['x1'], 100)
    yvals = source.data['y']
    normed = (yvals - yvals.min()) / (yvals.max()-yvals.min())
    ranged = normed * (ranges['y1'] - ranges['y0']) + ranges['y0']
    source.data.update({'x': xvals,  'y': ranged})
    push_notebook()

source = ColumnDataSource(data=dict(x=np.linspace(0, 10, 100), y=np.random.rand(100)*10))
p.circle('x', 'y', source=source)
    
p.x_range.callback = CustomJS(
        args=dict(source=source, x_range=p.x_range, y_range=p.y_range), code=jscode % 'normalize_points')
p.y_range.callback = CustomJS(
        args=dict(source=source, x_range=p.x_range, y_range=p.y_range), code=jscode % 'normalize_points')


show(p)